In [1]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [2]:
class NameToClass:
    def __init__(self):
        self.class_dict = {
            "Klein bottle": 0,
            "": 1,
            "RP^2": 2,
            "T^2": 3,
            "S^2": 4,
        }

    def __call__(self, data):
        # data.y = F.one_hot(torch.tensor(self.class_dict[data.name]),num_classes=5)
        data.y = torch.tensor(self.class_dict[data.name])
        return data


tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        NameToClass(),
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)

In [3]:
print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
# print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 1

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 1], y=4)
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


C:\Users\ernst\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'torsion_coefficients', 'n_vertices', 'betti_numbers', 'dimension', 'orientable', 'genus', 'face', 'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [4]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 562
Number of test graphs: 150


In [5]:
train_loader = DataLoader(
    train_dataset, batch_size=10
)  # ,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([4, 4, 0, 3, 4, 2, 2, 1, 4, 2])

In [6]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TAGConv, TransformerConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = TransformerConv(
            dataset.num_node_features, hidden_channels
        )
        self.conv2 = TransformerConv(hidden_channels, hidden_channels)
        self.conv3 = TransformerConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 5)

    def forward(self, batch):
        # 1. Obtain node embeddings
        x = self.conv1(batch.x, batch.edge_index)
        x = x.relu()
        x = self.conv2(x, batch.edge_index)
        x = x.relu()
        x = self.conv3(x, batch.edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch.batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): TransformerConv(1, 64, heads=1)
  (conv2): TransformerConv(64, 64, heads=1)
  (conv3): TransformerConv(64, 64, heads=1)
  (lin): Linear(in_features=64, out_features=5, bias=True)
)


In [7]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int(
            (pred == data.y).sum()
        )  # Check against ground-truth labels.
    return correct / len(
        loader.dataset
    )  # Derive ratio of correct predictions.


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 0.3327, Test Acc: 0.2933
Epoch: 002, Train Acc: 0.4751, Test Acc: 0.4533
Epoch: 003, Train Acc: 0.5089, Test Acc: 0.4933
Epoch: 004, Train Acc: 0.5125, Test Acc: 0.4600
Epoch: 005, Train Acc: 0.5996, Test Acc: 0.5667
Epoch: 006, Train Acc: 0.6637, Test Acc: 0.6200
Epoch: 007, Train Acc: 0.7046, Test Acc: 0.6067
Epoch: 008, Train Acc: 0.8167, Test Acc: 0.7867
Epoch: 009, Train Acc: 0.8345, Test Acc: 0.8000
Epoch: 010, Train Acc: 0.8310, Test Acc: 0.8000
Epoch: 011, Train Acc: 0.8185, Test Acc: 0.7600
Epoch: 012, Train Acc: 0.8114, Test Acc: 0.7667
Epoch: 013, Train Acc: 0.8363, Test Acc: 0.7933
Epoch: 014, Train Acc: 0.8363, Test Acc: 0.8000
Epoch: 015, Train Acc: 0.8363, Test Acc: 0.7933
Epoch: 016, Train Acc: 0.8381, Test Acc: 0.8000
Epoch: 017, Train Acc: 0.8399, Test Acc: 0.8000
Epoch: 018, Train Acc: 0.8381, Test Acc: 0.8000
Epoch: 019, Train Acc: 0.8399, Test Acc: 0.8000
Epoch: 020, Train Acc: 0.8256, Test Acc: 0.7933
Epoch: 021, Train Acc: 0.8381, Test Acc: